###Importando bibliotecas

In [ ]:
pip install msal

Python interpreter will be restarted.
Requirement already satisfied: msal in /databricks/python3/lib/python3.8/site-packages (1.20.0)
Requirement already satisfied: requests<3,>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from msal) (2.25.1)
Requirement already satisfied: cryptography<41,>=0.6 in /databricks/python3/lib/python3.8/site-packages (from msal) (39.0.0)
Requirement already satisfied: PyJWT[crypto]<3,>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from msal) (2.6.0)
Requirement already satisfied: cffi>=1.12 in /databricks/python3/lib/python3.8/site-packages (from cryptography<41,>=0.6->msal) (1.14.5)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi>=1.12->cryptography<41,>=0.6->msal) (2.20)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->msal) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->msal) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->msal) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->msal) (4.0.0)
WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d3b4c20c-8cf3-4c34-8477-d2f7cbf9cdb7/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [ ]:
import msal
from pyspark.sql.functions import *

##Conexão com o Azure Blob Storage

In [ ]:
# Criação da chave (variável), que permite a conexão com o Blob Storage
chave = dbutils.secrets.get(scope = 'XXXXXXX', key = 'XXXXXXXX')

###Extração de dados do container do Blob Storage para o Databricks

In [ ]:
# Conta e container necessário para acesso 
conta_armazenamento = 'XXXXXXX'
container = 'XXXXXXX'

###Criação do Mount point (ponto de montagem)

In [ ]:
# Monta o diretório de origem especificado no DBFS(Databricks File System) no ponto de montagem especificado.
try:
    dbutils.fs.mount(
      source = f"wasbs://{container}@{conta_armazenamento}.blob.core.windows.net", #Caminho de origem para a criação do mount
      mount_point = f"/mnt/{container}", # Nome do mount
      extra_configs = {f"fs.azure.account.key.{conta_armazenamento}.blob.core.windows.net": chave} # config + chave de acesso ao Blob
        )
except:
    pass


In [ ]:
# Verificando se o ponto de montagem foi criado
try:
    display(dbutils.fs.ls('/mnt/thiago-wilian/museus.parquet'))
    display(dbutils.fs.ls('/mnt/thiago-wilian/eventos.parquet'))
    
except:
    print('Ponto de montagem não localizado no DBFS')

path,name,size
dbfs:/mnt/thiago-wilian/museus.parquet,museus.parquet,1016297


path,name,size
dbfs:/mnt/thiago-wilian/eventos.parquet,eventos.parquet,699326


###Extração de dados do container do Blob Storage para o Databricks

#### Museus

In [ ]:
# Carregamento do arquivo csv para o DBFS e criação do DF
dados = '/mnt/thiago-wilian/museus.parquet'

df_museus = spark.read.load(path = dados, format = 'parquet', header = True, inferSchema = True)

In [ ]:
display(df_museus)

Id,Nome,Desc_curta,Regiao,Estado,Municipio,Endereco,Horario,Ingresso_cobrado,Site,Telefone_pub,Email_pub,Acessibilidade,Acess_fisica,Acess_visual,Latitude,Longitude
68,Memorial Juarez Barroso,"O Memorial Juarez Barroso abrigará espaço para o funcionamento de uma galeria que possa receber exposições diversas, sediará um núcleo de apoio a criação artística e um Memorial com fotos históricas dos povos das serras cearences.",Nordeste,CE,Guaramiranga,"Avenida Vicente Soares, s/n, Ao lado do Teatro Municipal, Centro, 62766000, Guaramiranga, CE",De 08h às 12 e de 13h às 17h.,Não,Não informado,(85) 99707-7232,cultura@guaramiranga.ce.gov.br,Não,Não possui,Não possui,-4.2618162,-38.9330716
6077,Parque Estadual Matas do Segredo,"Este é outro remanescente de cerrado protegido dentro do perímetro urbano de Campo Grande. Criado em 2000, o Parque Estadual das Matas do Segredo protege em seus 177,88 hectares as inúmeras nascentes que formam o Córrego Segredo .",Centro-Oeste,MS,Campo Grande,"Rua Josefina Mingarelli, s/n , Jardim Presidente, 79015-350, Campo Grande, MS",De terça a sexta-feira das 8h às 12h e das 14h às 17h.,Não,Não informado,(67) 3318-5719,cadastro@mma.gov.br,Não,Não possui,Não possui,-20.3994566,-54.585565
6078,Museu da Escola Politécnica da Universidade Federal do Rio de Janeiro,"Museu da Escola Politécnica O Museu da Escola Politécnica, inaugurado em 1977, tem o propósito de preservar a memória da engenharia brasileira. Seu acervo de mais de 600 itens ilustra os momentos político, cultural e econômico vivenciados pela ciência e tecnologia nacionais em toda a sua história (sobretudo no século XIX e início do XX).",Sudeste,RJ,Rio de Janeiro,"UFRJ - Avenida Athos da Silveira Ramos 149, Prédio do Centro de Tecnologia - Bloco A - 2º andar - Cidade Universitária, Ilha do Fundão, 21949-909, Rio de Janeiro, RJ",De segunda a sexta-feira das 8h às 17h.,Não,http://www.poli.ufrj.br/museu,(21) 3938-7723,museu_da_poli@poli.ufrj.br,Não,Não possui,Não possui,-22.904424,-43.228568
6079,Museu Municipal Marechal Bormann,Museu municipal,Sul,SC,Xanxerê,"Rua Doutor José de Miranda Ramos, 421 , Centro, 89820-000, Xanxerê, SC",Não informado,Não,Não informado,(49) 3981-8106,cultura@xanxere.sc.gov.br,Sim,Não informado,Não possui,-26.8775867,-52.4025932
6080,Memorial Alemão - Bosque Alemão,"O Bosque Alemão, antiga chácara da família Schaffer com 38 mil m² de área, foi inaugurado em 1996 na cidade de Curitiba, capital do estado brasileiro do Paraná. Localiza-se no bairro Vista Alegre e é formado por mata nativa densa.",Sul,PR,Curitiba,"Rua Francisco Schaffer s/n, Jardim Schaffer, Vista Alegre, 80820-200, Curitiba, PR",Não informado,Não,http://www.curitiba.pr.gov.br,Não informado,parquesepracas@smma.curitiba.pr.gov.br,Não,Não informado,Não informado,-25.406096,-49.285545
6081,Museu Municipal de Caxias do Sul,"Criado por meio da Lei Municipal número 02, de 03 de outubro de 1947, o Museu Municipal de Caxias do Sul surge, no tempo presente, como um dos espaços de memória mais representativos da localidade.",Sul,RS,Caxias do Sul,"Rua Visconde de Pelotas 586, Centro, 95020-180, Caxias do Sul, RS","Terça a sexta-feira, das 9h às 17h, e aos sábados, das 11h às 17h. horários aos domingos: consultar pelos telefones (54) 3221.2423 ou 3215.4445",Não,http://www.caxias.rs.gov.br/cultura/,(54) 3221-2423,museumunicipal@caxias.rs.gov.br,Sim,Não informado,Não informado,-29.1634,-51.179675
6083,Museu Histórico Municipal de São Joaquim - Espaço Assis Chateaubriand,Museu Histórico Municipal.,Sul,SC,São Joaquim,"Rua Major Jacinto Goulart 168, Centro, 88600-000, São Joaquim, SC",De terça a sexta-feira das 9h às 11h e das 13h30 às 17h. grupos grandes devem agendar a visita.,Não,Não informado,(49) 3233-2798,museusaojoaquim2018@gmail.com,Não,Não possui,Não possui,-28.290464,-49.937584
6084,Museu Histórico e Arqueológico Chico Ribeiro e Casa da Cultura Tias Polveiras,Museu Histórico e Arqueológico Chico Ribeiro e Casa da Cultura Tias Polveiras.,Sudeste,MG,Tupaciguara,"Praça do R

##### Converção do tipo dos dados das colunas 'Id', 'Latitude' e 'Longitude'

In [ ]:
df_museus = df_museus.withColumn('Id',df_museus['Id'].cast('integer'))\
            .withColumn('Latitude',df_museus['Latitude'].cast('float'))\
            .withColumn('Longitude',df_museus['Longitude'].cast('float'))\
            .withColumnRenamed('Site','Sites')
            

#### Eventos

In [ ]:
# Carregamento do arquivo csv para o DBFS e criação do DF
dados = '/mnt/thiago-wilian/eventos.parquet'

df_eventos = spark.read.load(path = dados, format = 'parquet', header = True, inferSchema = True)

##### Conversão do tipo de dado da coluna 'Id', 'Inicio' e 'Fim'

In [ ]:
df_eventos = df_eventos.withColumn('Id',df_eventos['Id'].cast('integer'))\
            .withColumn('Inicio', to_date('Inicio', 'dd/MM/yyyy'))\
            .withColumn('Fim', to_date('Fim', 'dd/MM/yyyy'))
            

In [ ]:
display(df_eventos.head(5))

Id Id_museu Nome_evento Curta_desc Longa_desc Subtitulo Preco Inicio Fim Hora_inicio Hora_fim Descript Class_etaria Frequencia Telefone_pub Sites Trad_libras Desc_sonora Obs 1325 202940 Mostra | Diálogo Design: Polônia Brasil Com a curadoria de Magda Kochanowska, Gabriel Patrocinio e Ewa Solarz, a exposição apresenta um diálogo entre a produção brasileira e a polonesa através do Design – gráfico e de produto. Após uma temporada em Brasília e outra no Rio de Janeiro, a mostra chega em São Paulo no Museu da Casa Brasileira e conta também com uma exposição inédita peças de Jorge Zalszupin. Dois países, duas culturas e duas histórias se unem para promover um diálogo entre design histórico e contemporâneo. Esse é o intuito da exposição ‘Diálogo Design: Polônia Brasil’, que chega a São Paulo no dia 5 de setembro, terça-feira, às 19h, no Museu da Casa Brasileira, instituição da Secretaria da Cultura do Estado de São Paulo, em parceria com The Spirit of Poland e Instituto Adam Mickiewicz. A mostra, que tem curadoria das polonesas Magda Kochanowska e Ewa Solarz e do brasileiro Gabriel Patrocínio, segue o conceito de estabelecer paralelos entre os designers da Polônia e do Brasil. O curador explica que foi a partir da constatação de semelhanças entre a atividade de projetar dos dois países que surgiu a ideia da exposição, e enfatiza o quanto suas culturas se aproximam quando o assunto é design. “Obviamente, existem diferenças históricas e outras influências, que também refletem na produção contemporânea dessas nações, que nos levaram a acreditar que essa justaposição poderia oferecer um rico panorama de atuação e propor, ao mesmo tempo, caminhos para uma eventual colaboração entre os dois países”, comenta Gabriel. A abertura terá início às 19h, com uma mesa-redonda moderada pelo curador Gabriel Patrocínio e seus convidados Rodrigo Brenner, do Furf Design; Victor Papapelo, do Grande Circular; Magdalena Kochanowska; curadora da mostra; e Weronika Rochacka Gagliardi; do The Spirit of Poland. Seguida por um coquetel, às 20h30. “A mostra ‘Diálogo Design: Polônia Brasil’ demonstra o objetivo do MCB em dar continuidade às parcerias internacionais, que asseguram a maior heterogeneidade de nossa programação expositiva”, afirma Giancarlo Latorraca, diretor técnico do museu. “É como se a casa brasileira se abrisse à casa polonesa, numa atitude cosmopolita típica de uma metrópole como São Paulo, trazendo para dentro os objetos de uma nação que contribui para a nossa formação multicultural”, complementa Gabriel Patrocínio, curador da exposição. A exposição ‘Diálogo Design: Polônia Brasil’ reúne tradição e inovação em 20 cartazes e 25 peças icônicas de mobiliário, vidro e porcelana. Além disso, cinco designers gráficos poloneses e cinco brasileiros foram convidados para retratar dois cartazes, um representando a sua nação de origem e o outro com a sua visão do segundo país da mostra – cartazes que serão distribuídos, gratuitamente, no dia da abertura da exposição. Jorge Zalszupin - novidade da ‘Diálogo Design: Polônia Brasil’ no MCB Após uma temporada em Brasília e outra no Rio de Janeiro, a exposição ‘Diálogo Design: Polônia Brasil’ inaugura em São Paulo, no Museu da Casa Brasileira, em versão ampliada: incorpora um recorte com peças de Jorge Zalszupin, arquiteto polonês naturalizado brasileiro e conhecido pela importância da sua contribuição na construção da identidade do design moderno brasileiro. Essa seleção, em homenagem ao designer, será apresentada pela primeira vez na capital paulista. Por meio dessa pequena mostra, intitulada “JZ: O diálogo personificado", os visitantes poderão conhecer com riqueza de detalhes do trabalho do designer, disposto em duas salas. “Zalszupin personifica o diálogo que buscamos para representar a exposição como um todo. Reunimos um pequeno, mas significativo - quase icônico - conjunto de nove peças projetadas por ele. Isso mereceu um segmento especial”, explica o curador, Gabriel Patrocínio. Programação Além da mos

###Conexão com o SQL SERVER

In [ ]:
# Id do locatário/inquilino: Localizado em REGISTROS DE ATIVIDADES DO AZURE ACTIVE DIRETORY -> app-user-databricks
tenant_id = "XXXXXXXXXXXXXXXX"
msal_authority = "https://login.windows.net/" + tenant_id

# Url do recurso do Database
resource_url = "https://database.windows.net/.default"

#Recupera valores de chaves guardadas no Key Vault
service_principal_id = dbutils.secrets.get(scope = "XXXXXXXXXXXXXX", key = "XXXXXXXXX")
service_principal_secret = dbutils.secrets.get(scope = "XXXXXXXXX", key = "XXXXXXXXXX")

# Criando a Instância que será usada durante todo o ciclo do aplicativo
instance = msal.ConfidentialClientApplication(service_principal_id, service_principal_secret, msal_authority)

# Enviando solicitação ao Azure Active Diretory para obter um token
token = instance.acquire_token_for_client(resource_url)
access_token = token["access_token"]



jdbcHostname = "XXXXXXXXX.database.windows.net"
jdbcDataBase = "XXXXXXXXX"
jdbcPort = XXXXX
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDataBase}"

###Gravação dos dados na Tabela

In [ ]:
df_museus.write.format('jdbc')\
    .mode('overwrite')\
    .option('url', jdbcUrl)\
    .option('dbtable', '[STAGE_thiago_wilian4].[STAGE_museus]')\
    .option('accessToken', access_token)\
    .save()

df_eventos.write.format('jdbc')\
    .mode('overwrite')\
    .option('url', jdbcUrl)\
    .option('dbtable', '[STAGE_thiago_wilian4].[STAGE_eventos]')\
    .option('accessToken', access_token)\
    .save()

###Excluindo o Mount point

In [ ]:
# Exclui o ponto de montagem do DBFS.
try:
    dbutils.fs.unmount("/mnt/thiago-wilian")
except:
    print('Ponto de montagem inexistente')
else: 
    print('Desmonte realizado com sucesso')

/mnt/thiago-wilian has been unmounted.
Desmonte realizado com sucesso